In [73]:
import numpy as np
from pyscf import gto, scf, cc
from scipy.linalg import block_diag
from BSE_Helper import spinor_one_and_two_e_int

np.set_printoptions(precision=6, suppress=True, linewidth=100000)
eV_to_Hartree = 0.0367493

def get_selfenergy_spatial(t2,oovv, goovv):
    selfener_occ_1 = np.einsum('ikab, jkab -> ij', oovv, t2, optimize="optimal")
    selfener_occ_2 = -np.einsum('ikab, jkba -> ij', oovv, t2, optimize="optimal")
    selfener_occ_3 = np.einsum('ikab, jkab -> ij', goovv, t2, optimize="optimal")
    selfener_occ_4 = np.einsum('ikba, jkba -> ij', goovv, t2, optimize="optimal")

    selfener_occ = 0.5*(selfener_occ_1 + selfener_occ_2 + selfener_occ_3 + selfener_occ_4)
    
    selfener_vir_1 = np.einsum('ijbc, ijac -> ab', oovv, t2, optimize="optimal")
    selfener_vir_2 = - np.einsum('ijbc, ijca -> ab', oovv, t2, optimize="optimal")
    selfener_vir_3 = np.einsum('ijbc, ijac -> ab', goovv, t2, optimize="optimal")
    selfener_vir_4 = np.einsum('ijcb, ijca -> ab', goovv, t2, optimize="optimal")

    selfener_vir = -0.5*(selfener_vir_1 + selfener_vir_2 + selfener_vir_3 + selfener_vir_4)
    return selfener_occ, selfener_vir


def build_fock_matrices_spatial(mol)-> tuple[np.ndarray,np.ndarray]:

    mf = scf.HF(mol)     
    mf.kernel()          
    F_ao = mf.get_fock()    
    C   = mf.mo_coeff        
    F_mo = C.T @ F_ao @ C   
    fock_occ = F_mo[:int(n_occ),:int(n_occ)]
    fock_vir = F_mo[int(n_occ):,int(n_occ):]

    return fock_occ, fock_vir

def build_gfock_spatial(t2, oovv, goovv, selfener_occ, selfener_vir, fock_occ, fock_vir) -> tuple[np.ndarray,np.ndarray]:
    occ_selfeng, vir_selfeng = get_selfenergy_spatial(t2,oovv, goovv)
    fock_occ, fock_vir = build_fock_matrices_spatial(mol) 
    
    F_ij = occ_selfeng + fock_occ
    F_ab = vir_selfeng + fock_vir 

    return F_ij, F_ab

def build_bse(F_ij, F_ab, n_occ, n_vir, goovv, ovvo, t2) -> tuple[np.ndarray,np.ndarray,np.ndarray,np.ndarray,np.ndarray]:

    F_abij = np.einsum('ab, ij -> iajb', F_ab, np.identity(n_occ),optimize='optimal')
    F_ijab = np.einsum('ij, ab -> iajb', F_ij, np.identity(n_vir),optimize='optimal')

    nspincase = 2
    H_bse = np.zeros((n_occ,n_vir,n_occ,n_vir,nspincase))
    
    for i in range(2):
        H_bse[:,:,:,:,i] = F_abij - F_ijab + np.einsum("iabj->iajb", ovvo,optimize='optimal')

        for ispina in range(2):
            for ispini in range (2):
                if ispina == ispini:
                    # ispincase = 1  or 2
                    term1 = np.einsum("ikbc, ijab -> iajb", oovv,t2,optimize="optimal")
                    term2 = np.einsum("ikbc, ijba -> iajb", oovv,t2,optimize="optimal")
                    term3 = - np.einsum("ikbc, jkac -> iajb", goovv,t2,optimize="optimal")
                    
                    H_bse[:,:,:,:,i] += term1 + term2 + term3
    
                else:
                    # ispincase = 3 or 4
                    H_bse[:,:,:,:,i] += - np.einsum("ikbc, jkac -> iajb", oovv,t2,optimize="optimal") 
    return H_bse

if __name__ == "__main__":

    # Define Molecule to calculate amplitudes and mo for
    mol = gto.M(atom="H 0.00 0.00 0.00; H 0.00 0.00 2.00",
                basis='cc-pVTZ',
                spin=0,
                symmetry=False,
                unit="Bohr")

    
    # get molecular orbitals and t2 amplitudes
    myhf = mol.HF.run() 
    mycc = cc.BCCD(myhf,conv_tol_normu=1e-8).run()
    mo = mycc.mo_coeff
    t2 = mycc.t2

    # spatial orbital numbers
    n_occ = t2.shape[0]
    n_vir = t2.shape[2]

    eri_ao = mol.intor('int2e') # two e integral
    eri_ao = np.einsum("pqrs->prqs", eri_ao, optimize="optimal")    # Convert to Physicist's notation
    anti_eri_ao = eri_ao - np.einsum("prqs->prsq", eri_ao, optimize='optimal')

    core_spatialorbs = mo[:, 0].reshape(-1,1)
    vir_spatialorbs = mo[:, 1:]

    # Build the required anti-symmetrised orbitals
    oovv = np.einsum("pi,qj,pqrs,ra,sb->ijab", core_spatialorbs, core_spatialorbs, anti_eri_ao, vir_spatialorbs, vir_spatialorbs, optimize="optimal")
    ovvo = np.einsum("pi,qa,pqrs,rb,sj->iabj", core_spatialorbs, vir_spatialorbs, anti_eri_ao, vir_spatialorbs, core_spatialorbs,optimize="optimal") 
    goovv = np.einsum("pi,qj,pqrs,ra,sb->ijab", core_spatialorbs, core_spatialorbs, eri_ao, vir_spatialorbs, vir_spatialorbs, optimize="optimal")

    # build self energy
    selfener_occ, selfener_vir = get_selfenergy_spatial(t2,oovv, goovv) # n_occ x n_occ, n_vir x n_vir

    # build fock matrix
    fock_occ, fock_vir = build_fock_matrices_spatial(mol) # n_occ x n_occ, n_vir x n_vir

    # build gfock
    F_ij = selfener_occ + fock_occ
    F_ab = selfener_vir + fock_vir 

    # (n_occ,n_vir,n_occ,n_vir,nspincase)
    hbse = build_bse(F_ij, F_ab, n_occ, n_vir, goovv, ovvo, t2)
    
    #H_bse = H_bse.reshape((n_occ*n_vir,n_occ*n_vir))
    #e, v = np.linalg.eig(hbse)


    

converged SCF energy = -1.09108365450833
E(CCSD) = -1.136171675463324  E_corr = -0.04508802095499804
E(CCSD) = -1.136171675078332  E_corr = -0.0457166106717859
E(CCSD) = -1.136171555724961  E_corr = -0.04569116974162639
E(CCSD) = -1.136171657947945  E_corr = -0.04569139327446291
E(CCSD) = -1.13617168257757  E_corr = -0.04569146490871473
E(CCSD) = -1.136171681247839  E_corr = -0.04569146472842286
E(CCSD) = -1.13617168422992  E_corr = -0.04569146465231588
E(CCSD) = -1.136171685777563  E_corr = -0.04569146318370622
E(CCSD) = -1.136171686724058  E_corr = -0.0456914624589354
E(CCSD) = -1.136171684947967  E_corr = -0.04569145130255949
E(CCSD) = -1.136171687891508  E_corr = -0.04569145556808186
E(CCSD) = -1.136171689861596  E_corr = -0.04569146126014119
E(CCSD) = -1.136171689297032  E_corr = -0.04569146117272486
E(CCSD) = -1.136171689089693  E_corr = -0.04569146113791447
E(CCSD) = -1.136171688990617  E_corr = -0.04569146115223236
E(CCSD) = -1.136171689152756  E_corr = -0.04569146191176564
con

In [74]:
np.diag(selfener_vir)/eV_to_Hartree

array([0.18234 , 0.041871, 0.165764, 0.050817, 0.050817, 0.019442, 0.00466 , 0.00466 , 0.012608, 0.016236, 0.021149, 0.008438, 0.008438, 0.002057, 0.002057, 0.011007, 0.000841, 0.000841, 0.003283, 0.003283, 0.002764, 0.002764, 0.001442, 0.00091 , 0.00091 , 0.000833, 0.001435])

In [75]:
selfener_occ/eV_to_Hartree

array([[-0.621664]])

In [76]:
hbse.shape

(1, 27, 1, 27, 2)